In [1]:
!pip install kfp

     |████████████████████████████████| 225 kB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 69.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 47.1 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 43 kB/s s eta 0:00:01
     |████████████████████████████████| 54 kB 4.1 MB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 82 kB 1.6 MB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 8.5 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-1.6.4-py3-none-any.whl size=307978 sha256=1393274678f3592b858fb67c421c33399cd6ae62cc2ebb1d9155d38312ed1513
  Stored in directory: /home/jupyter/.cache/pip/wheels/68/fb/ec/d4c6e6c3a85acbf75cf4a868faf8f2fe98b75973c223f8f10c
  Created wheel for docstring-parser: filename=docstring_parser-0.8.

In [103]:
%%writefile ./pipeline/iris_training_pipeline.py

"""KFP orchestrating BigQuery and Cloud AI Platform services."""

import os

from helper_components import evaluate_model
from helper_components import retrieve_best_run
from helper_components import custom_deploy
from helper_components import prepoc_split_dataset
from jinja2 import Template
import kfp
from kfp.components import func_to_container_op
from kfp.dsl.types import Dict
from kfp.dsl.types import GCPProjectID
from kfp.dsl.types import GCPRegion
from kfp.dsl.types import GCSPath
from kfp.dsl.types import String
from kfp.gcp import use_gcp_secret

# Defaults and environment settings
BASE_IMAGE = os.getenv('BASE_IMAGE')
TRAINER_IMAGE = os.getenv('TRAINER_IMAGE')
RUNTIME_VERSION = os.getenv('RUNTIME_VERSION')
PYTHON_VERSION = os.getenv('PYTHON_VERSION')
COMPONENT_URL_SEARCH_PREFIX = os.getenv('COMPONENT_URL_SEARCH_PREFIX')
USE_KFP_SA = os.getenv('USE_KFP_SA')

TRAINING_FILE_PATH = 'datasets/training/data.csv'
VALIDATION_FILE_PATH = 'datasets/validation/data.csv'
TESTING_FILE_PATH = 'datasets/testing/data.csv'

# Parameter defaults
SPLITS_DATASET_ID = 'splits'
HYPERTUNE_SETTINGS = """
{
    "hyperparameters":  {
        "goal": "MAXIMIZE",
        "maxTrials": 6,
        "maxParallelTrials": 3,
        "hyperparameterMetricTag": "accuracy",
        "enableTrialEarlyStopping": True,
        "params": [
            {
                "parameterName": "max_iter",
                "type": "DISCRETE",
                "discreteValues": [90, 100]
            },
            {
                "parameterName": "alpha",
                "type": "DOUBLE",
                "minValue": 0.001,
                "maxValue": 0.004,
                "scaleType": "UNIT_LINEAR_SCALE"
            }
        ]
    }
}
"""

# Create component factories
component_store = kfp.components.ComponentStore(
    local_search_paths=None, url_search_prefixes=[COMPONENT_URL_SEARCH_PREFIX])

prepoc_split_op = func_to_container_op(prepoc_split_dataset, base_image=BASE_IMAGE)
mlengine_train_op = component_store.load_component('ml_engine/train')
mlengine_deploy_op = component_store.load_component('ml_engine/deploy')
retrieve_best_run_op = func_to_container_op(
    retrieve_best_run, base_image=BASE_IMAGE)
evaluate_model_op = func_to_container_op(evaluate_model, base_image=BASE_IMAGE)
custom_deploy_op = func_to_container_op(custom_deploy, base_image=BASE_IMAGE)


@kfp.dsl.pipeline(
    name='Iris Classifier Training',
    description='The pipeline training and deploying the Iris classifierpipeline_yaml'
)
def iris_train(project_id,
                    region,
                    gcs_root,
                    dataset_id,
                    evaluation_metric_name,
                    evaluation_metric_threshold,
                    model_id,
                    version_id,
                    replace_existing_version,
                    hypertune_settings=HYPERTUNE_SETTINGS,
                    dataset_location='US'):
    """Orchestrates training and deployment of an sklearn model."""

    splited_prepoc_dataset = prepoc_split_op(gcs_root)
   
    # Tune hyperparameters
    tune_args = [
        '--training_dataset_path',
        splited_prepoc_dataset.outputs['training_file_path'],
        '--validation_dataset_path',
        splited_prepoc_dataset.outputs['validation_file_path'], '--hptune', 'True'
    ]

    job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir/hypertune',
                                kfp.dsl.RUN_ID_PLACEHOLDER)

    hypertune = mlengine_train_op(
        project_id=project_id,
        region=region,
        master_image_uri=TRAINER_IMAGE,
        job_dir=job_dir,
        args=tune_args,
        training_input=hypertune_settings)

    # Retrieve the best trial
    get_best_trial = retrieve_best_run_op(
            project_id, hypertune.outputs['job_id'])

    # Train the model on a combined training and validation datasets
    job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir', kfp.dsl.RUN_ID_PLACEHOLDER)

    train_args = [
        '--training_dataset_path',
        splited_prepoc_dataset.outputs['training_file_path'],
        '--validation_dataset_path',
        splited_prepoc_dataset.outputs['validation_file_path'], '--alpha',
        get_best_trial.outputs['alpha'], '--max_iter',
        get_best_trial.outputs['max_iter'], '--hptune', 'False'
    ]

    train_model = mlengine_train_op(
        project_id=project_id,
        region=region,
        master_image_uri=TRAINER_IMAGE,
        job_dir=job_dir,
        args=train_args)

    # Evaluate the model on the testing split
    eval_model = evaluate_model_op(
        testing_file_path=str(splited_prepoc_dataset.outputs['testing_file_path']),
        model_path=str(train_model.outputs['job_dir']),
        metric_name=evaluation_metric_name)

    # Deploy the model if the primary metric is better than threshold
    with kfp.dsl.Condition(eval_model.outputs['metric_value'] > evaluation_metric_threshold):
        custom_deploy_op(
        model_uri=str(train_model.outputs['job_dir']),
        project_id=project_id,
        model_id=model_id,
        version_id=version_id)

    # Configure the pipeline to run using the service account defined
    # in the user-gcp-sa k8s secret
    if USE_KFP_SA == 'True':
        kfp.dsl.get_pipeline_conf().add_op_transformer(
              use_gcp_secret('user-gcp-sa'))

Overwriting ./pipeline/iris_training_pipeline.py


In [3]:
!gsutil ls

gs://artifacts.robotic-tide-284315.appspot.com/
gs://robotic-tide-284315-kubeflowpipelines-default/
gs://robotic-tide-284315_cloudbuild/


In [116]:
REGION = 'us-central1'
ENDPOINT = '74b539b8a4259f1d-dot-us-central1.pipelines.googleusercontent.com' # TO DO: REPLACE WITH YOUR ENDPOINT
ARTIFACT_STORE_URI = 'gs://robotic-tide-284315-kubeflowpipelines-default'  # TO DO: REPLACE WITH YOUR ARTIFACT_STORE NAME 
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [63]:
IMAGE_NAME='trainer_image'
TAG='latest'
TRAINER_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [64]:
!gcloud builds submit --timeout 15m --tag $TRAINER_IMAGE trainer_image

Creating temporary tarball archive of 4 file(s) totalling 5.6 KiB before compression.
Uploading tarball of [trainer_image] to [gs://robotic-tide-284315_cloudbuild/source/1626369533.646846-a1d36366220d47f48ef36ea716c804b1.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/robotic-tide-284315/locations/global/builds/254af586-c711-455b-b054-a59abef9d959].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/254af586-c711-455b-b054-a59abef9d959?project=678865744419].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "254af586-c711-455b-b054-a59abef9d959"

FETCHSOURCE
Fetching storage object: gs://robotic-tide-284315_cloudbuild/source/1626369533.646846-a1d36366220d47f48ef36ea716c804b1.tgz#1626369534027063
Copying gs://robotic-tide-284315_cloudbuild/source/1626369533.646846-a1d36366220d47f48ef36ea716c804b1.tgz#1626369534027063...
/ [1 files][  1.6 KiB/  1.6 KiB]                                                
Operat

In [8]:
IMAGE_NAME='base_image'
TAG='latest'
BASE_IMAGE='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

In [9]:
!gcloud builds submit --timeout 15m --tag $BASE_IMAGE base_image

Creating temporary tarball archive of 2 file(s) totalling 244 bytes before compression.
Uploading tarball of [base_image] to [gs://robotic-tide-284315_cloudbuild/source/1626363743.727561-09685699bdcc4c5ba9e537f6da00454a.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/robotic-tide-284315/locations/global/builds/f1d6279d-8481-4a5f-90a9-9c322098bdf2].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/f1d6279d-8481-4a5f-90a9-9c322098bdf2?project=678865744419].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "f1d6279d-8481-4a5f-90a9-9c322098bdf2"

FETCHSOURCE
Fetching storage object: gs://robotic-tide-284315_cloudbuild/source/1626363743.727561-09685699bdcc4c5ba9e537f6da00454a.tgz#1626363744072585
Copying gs://robotic-tide-284315_cloudbuild/source/1626363743.727561-09685699bdcc4c5ba9e537f6da00454a.tgz#1626363744072585...
/ [1 files][  284.0 B/  284.0 B]                                                
Operati

In [10]:
USE_KFP_SA = False

COMPONENT_URL_SEARCH_PREFIX = 'https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/'
RUNTIME_VERSION = '1.15'
PYTHON_VERSION = '3.7'

%env USE_KFP_SA={USE_KFP_SA}
%env BASE_IMAGE={BASE_IMAGE}
%env TRAINER_IMAGE={TRAINER_IMAGE}
%env COMPONENT_URL_SEARCH_PREFIX={COMPONENT_URL_SEARCH_PREFIX}
%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}

env: USE_KFP_SA=False
env: BASE_IMAGE=gcr.io/robotic-tide-284315/base_image:latest
env: TRAINER_IMAGE=gcr.io/robotic-tide-284315/trainer_image:latest
env: COMPONENT_URL_SEARCH_PREFIX=https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/
env: RUNTIME_VERSION=1.15
env: PYTHON_VERSION=3.7


### Deploy the pipeline package

In [122]:
!dsl-compile --py pipeline/iris_training_pipeline.py --output iris_training_pipeline.yaml

In [123]:
PIPELINE_NAME='iris_continuous_training_v3'

!kfp --endpoint $ENDPOINT pipeline upload \
-p $PIPELINE_NAME \
iris_training_pipeline.yaml

/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 74b539b8a4259f1d-dot-us-central1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
Pipeline Details
------------------
ID           7b6732b9-0d5f-4ae5-bc8e-c3ea0efd6cab
Name         iris_continuous_training_v3
Description
Uploaded at  2021-07-15T21:37:42+00:00
+-----------------------------+--------------------------------------------------+
| Parameter Name              | Default Value                                    |
+=============================+==================================================+
| project_id                  |                                                  |
+-----------------------------+--------------------------------------------------+
| region                      |                                                  |
+-----------------------------+-------------------------------------------

In [124]:
!kfp --endpoint $ENDPOINT pipeline list

/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 74b539b8a4259f1d-dot-us-central1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
+--------------------------------------+------------------------------------------------+---------------------------+
| Pipeline ID                          | Name                                           | Uploaded at               |
+======================================+================================================+===========================+
| 7b6732b9-0d5f-4ae5-bc8e-c3ea0efd6cab | iris_continuous_training_v3                    | 2021-07-15T21:37:42+00:00 |
+--------------------------------------+------------------------------------------------+---------------------------+
| 70199f59-dbd2-4777-88fc-f489324df6dd | iris_continuous_training_v2                    | 2021-07-15T21:06:11+00:00 |
+--------------------------------------+--

In [125]:
PIPELINE_ID='7b6732b9-0d5f-4ae5-bc8e-c3ea0efd6cab' # TO DO: REPLACE WITH YOUR PIPELINE ID 

In [117]:
EXPERIMENT_NAME = 'Iris_Classifier_Training'
RUN_ID = 'Run_001'
DATASET_ID = 'splits'
EVALUATION_METRIC = 'accuracy'
EVALUATION_METRIC_THRESHOLD = '0.69'
MODEL_ID = 'iris_classifier'
VERSION_ID = 'v01'
REPLACE_EXISTING_VERSION = 'True'

GCS_STAGING_PATH = '{}/staging'.format(ARTIFACT_STORE_URI)

In [126]:
!kfp --endpoint $ENDPOINT run submit \
-e $EXPERIMENT_NAME \
-r $RUN_ID \
-p $PIPELINE_ID \
project_id=$PROJECT_ID \
gcs_root=$GCS_STAGING_PATH \
region=$REGION \
dataset_id=$DATASET_ID \
evaluation_metric_name=$EVALUATION_METRIC \
evaluation_metric_threshold=$EVALUATION_METRIC_THRESHOLD \
model_id=$MODEL_ID \
version_id=$VERSION_ID \
replace_existing_version=$REPLACE_EXISTING_VERSION

/opt/conda/lib/python3.7/site-packages/kfp/_client.py:182: UserWarning: The host 74b539b8a4259f1d-dot-us-central1.pipelines.googleusercontent.com does not contain the "http" or "https" protocol. Defaults to "https".
  ' Defaults to "https".' % host)
+--------------------------------------+---------+----------+---------------------------+
| run id                               | name    | status   | created at                |
+======================================+=========+==========+===========================+
| ed322b00-6e5c-407e-b667-b0a21ffbf451 | Run_001 |          | 2021-07-15T21:38:12+00:00 |
+--------------------------------------+---------+----------+---------------------------+


In [18]:
!echo $ENDPOINT

74b539b8a4259f1d-dot-us-central1.pipelines.googleusercontent.com
